## Simple benchmark using LGB

Benchamrk with LGB item_cnt_month clipped between o to 20, validation on month before prediction (33)
Final model is built using months upto 32 and final prediction is done on test data using that model.

In [1]:
import os
import sys
import pandas as pd

In [2]:
sys.path.insert(0, os.path.abspath('/home/jupyter/kaggle/predict_future_sales/src/'))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import common.com_util as util
import munging.process_data as process_data
import config.constants as constants
import cv.cv_util as cv
import modeling.train_util as model

In [5]:
logger = util.get_logger('sub_2')

In [6]:
train_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/train_all_merged.feather')
test_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/test_all_merged.feather')

train_features = ['shop_id', 'item_id', 'date_block_num', 'item_category_id', 'item_cnt_month']
train_df = train_df[train_features]

test_features = ['ID', 'shop_id', 'item_id', 'date_block_num', 'item_category_id']
test_df = test_df[test_features]

sample_submission = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/submission_processed.feather')

test = test_df.drop(['ID'], axis='columns')

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [7]:
TARGET = 'item_cnt_month'
ID = 'ID'
SEED = 42

training_months = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
holdout_months = [33]

training, validation = cv.get_data_splits_by_date_block(logger, train_df, 
                                                        train_months=training_months, 
                                                        validation_months=holdout_months)
# Clip the item_cnt_month within 20
training.item_cnt_month.clip(lower=0, upper=20, inplace=True)
validation.item_cnt_month.clip(lower=0, upper=20, inplace=True)

[INFO]2020-09-08 05:38:00,429:sub_2:Splitting the data into train and holdout based on months...
[INFO]2020-09-08 05:38:00,430:sub_2:Training months [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
[INFO]2020-09-08 05:38:00,431:sub_2:Validation months [33]
[INFO]2020-09-08 05:38:01,556:sub_2:Shape of the training data (10675678, 5) 
[INFO]2020-09-08 05:38:01,557:sub_2:Shape of the validation data (238172, 5)


/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/generic.py:7415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(result)


In [8]:
lgb_params = {
                'objective' : 'root_mean_squared_error',
                'boosting_type' : 'gbdt',
                'learning_rate' : 0.1,
                'num_leaves' : 31,
                'tree_learner' : 'serial',
                'n_jobs' :-1,
                'seed' : SEED,
                'max_depth' : -1,
                'max_bin' : 255,
                'metric' : 'rmse',
                'verbose' : 100
                }

In [9]:
predictors = ['shop_id', 'item_id', 'date_block_num', 'item_category_id']

In [12]:
bst, validation_score, prediction = model.lgb_train_validate_on_holdout(logger=logger, training=training, 
                                                            validation=validation, predictors=predictors, 
                                                            target=TARGET, params=lgb_params, 
                                                            test_X=test, n_estimators=10000,
                                                            early_stopping_rounds=100)

[INFO]2020-09-08 05:45:03,777:sub_2:Training using LGB and validating on holdout
[INFO]2020-09-08 05:45:03,891:sub_2:Shape of train_X, train_Y, validation_X, validation_Y: (10675678, 4), (10675678,), (238172, 4), (238172,)
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 1.10506	valid_1's rmse: 1.03643
[200]	training's rmse: 1.08403	valid_1's rmse: 1.02596
[300]	training's rmse: 1.07122	valid_1's rmse: 1.02005
[400]	training's rmse: 1.06171	valid_1's rmse: 1.01604
[500]	training's rmse: 1.05444	valid_1's rmse: 1.01293
[600]	training's rmse: 1.0486	valid_1's rmse: 1.01056
[700]	training's rmse: 1.04425	valid_1's rmse: 1.00948
[800]	training's rmse: 1.04055	valid_1's rmse: 1.00662
[900]	training's rmse: 1.03737	valid_1's rmse: 1.0053
[1000]	training's rmse: 1.03474	valid_1's rmse: 1.00565
Early stopping, best iteration is:
[906]	training's rmse: 1.03719	valid_1's rmse: 1.00525
[INFO]2020-09-08 05:47:28,917:sub_2:Validation Score 1.0052455937767015
[INF

In [15]:
bst, validation_score

(<lightgbm.basic.Booster at 0x7fc9ed18fa90>, 1.0052455937767015)

In [16]:
prediction.shape

(214200,)

In [18]:
prediction

array([0.58300424, 0.4513639 , 0.18785308, ..., 0.10960865, 0.11074544,
       0.07246491])

In [19]:
test_df.shape

(214200, 5)

In [20]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [21]:
submission = pd.DataFrame({'ID': test_df.ID, 'item_cnt_month': prediction})

In [22]:
submission.shape

(214200, 2)